# Projeto Spark

A ser realizado em duplas.

Entrega: 6 de dezembro de 2022

## Introdução

Neste projeto vamos construir um classificador Naive-Bayes para determinar o sentimento de um comentário.

## Instalando o ambiente

O jeito mais simples de começar a trabalhar com Spark é instalar um container com tudo pronto! No site https://hub.docker.com/r/jupyter/pyspark-notebook vemos uma imagem Docker que já vem com `pyspark` e `jupyter lab`. Instale a imagem com o comando:

```bash
docker pull jupyter/pyspark-notebook
```

Vamos iniciar o ambiente de trabalho com o comando `docker run`. Para isso precisamos tomar alguns cuidados:

1) Temos que mapear nosso diretorio local de trabalho para um diretório interno do container, de modo que alterações feitas dentro do container (nesta pasta escolhida) sejam gravadas no nosso diretorio local. No container temos um usuário padrão com *username* `jovyan`. No *homedir* desse usuario temos uma pasta vazia `work`, que vai servir como local de mapeamento do nosso diretorio local de trabalho. Podemos então fazer esse mapeamendo com a opção `-v` do comando `docker run` da seguinte forma:

```bash
-v <diretorio>:/home/jovyan/work
```

onde `<diretorio>` representa seu diretorio local de trabalho.

2) Para acessar o `jupyter notebook` e o *dashboard* do Spark a partir do nosso *browser* favorito temos que abrir algumas portas do container com a opção `-p`. As portas são `8888` (para o próprio `jupyter notebook`) e `4040` (para o *dashboard* do Spark). Ou seja, adicionaremos às opções do `docker run`o seguinte:

```bash
-p 8888:8888 -p 4040:4040
```

Desta forma, ao acessar `localhost:8888` na nossa máquina, estaremos acessando o servidor Jupyter na porta 8888 interna do container.

3) Vamos iniciar o container no modo interativo, e vamos especificar que o container deve ser encerrado ao fechar o servidor Jupyter. Faremos isso com as opções `-it` e `-rm`

Portanto, o comando completo que eu uso na minha máquina Linux para iniciar o container é:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v `pwd`:/home/jovyan/work \
    jupyter/pyspark-notebook


```

Para facilitar a vida eu coloco esse comando em um arquivo `inicia.sh`. Engenheiros, façam do jeito que preferirem!

Agora abra esse notebook lá no container!


## Iniciando o Spark

Vamos iniciar o ambiente Spark. Para isso vamos:

1) Criar um objeto de configuração do ambiente Spark. Nossa configuração será simples: vamos especificar que o nome da nossa aplicação Spark é "Minha aplicação", e que o *master node* é a máquina local, usando todos os *cores* disponíveis. Aplicações reais de Spark são configuradas de modo ligeiramente diferente: ao especificar o *master node* passamos uma URL real, com o endereço do nó gerente do *cluster* Spark.

2) Vamos criar um objeto do tipo `SparkContext` com essa configuração

In [1]:
import pyspark

conf = pyspark.SparkConf()
conf.setAppName('Minha aplicação')
conf.setMaster('local[*]')

sc = pyspark.SparkContext(conf=conf)

In [2]:
#outras bibliotecas
import numpy as np

O `SparkContext` é a nossa porta de entrada para o cluster Spark, ele será a raiz de todas as nossas operações com o Spark.

In [3]:
sc

<SparkContext master=local[*] appName=Minha aplicação>

O link acima provavelmente não funcionará porque ele se refere à porta 4040 interna do container (portanto a URL está com endereço interno). Porém fizemos o mapeamento da porta 4040 interna para a porta 4040 externa, logo você pode acessar o *dashboard* do Spark no endereço http://localhost:4040

<center><img src="./spark_dashboard.png" width=800/></center>

## Lendo os dados

Vamos começar lendo o arquivo de reviews e gravando o resultado em formato pickle, mais amigável.

In [2]:
def parse_line(line):
    parts = line[1:-1].split('","')
    sentiment = int(parts[0])
    title = parts[1].replace('""', '"')
    body = parts[2].replace('""', '"')
    return (sentiment, title, body)

rdd = sc.textFile('train.csv').map(parse_line)

Agora vamos gravar no formato pickle, para facilitar os trabalhos futuros. Após gravar o arquivo, não mais rode as células desta primeira etapa!

In [ ]:
rdd.saveAsPickleFile('reviews.pickle')

## Um classificador Naive-Bayes

Vamos ler o arquivo pickle gravado anteriormente:

In [4]:
rdd = sc.pickleFile('reviews.pickle')

Agora, complete as tarefas em sequencia para construir o classificador Naive-Bayes:

### Fase 1

#### Tarefa

Construa uma função que recebe um RDD no formato do RDD original e retorna um RDD no qual cada item é um par (palavra, contagem).

In [5]:
def separa_em_palavras(item):
    titulo = item[1]
    corpo = item[2]
    
    return titulo.split() + corpo.split()

def palavra_contagem(rdd):
    operacao = rdd.flatMap(separa_em_palavras).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)
    return operacao

In [6]:
new = palavra_contagem(rdd)

In [7]:
new.take(10)

[('out', 667825),
 ('like', 930710),
 ('heart-rendering', 20),
 ('family', 62599),
 ('book', 1398748),
 ('let', 73362),
 ('angery.', 5),
 ('purposes.', 1624),
 ('wasting', 8540),
 ('When', 119667)]

#### Tarefa

Construa uma função que recebe o RDD (palavra, contagem) construido anteriormente e retorna um RDD no qual cada item é um par (palavra, $\log_{10}\left(c \, / \, T\right)$), onde $c$ é a contagem daquela palavra e $T$ é a soma das contagens de palavra.

In [8]:
total_ = new.reduce(lambda x, y: ("tudo", x[1] + y[1]))

In [9]:
def logaritmo(rdd, T):
    operacao = rdd.map(lambda x: (x[0], np.log10(x[1]/T[1])))
    return operacao

In [10]:
new_2 = logaritmo(new, total_)

In [11]:
new_2.take(3)

[('out', -2.6263333876130455),
 ('like', -2.4821816802222187),
 ('heart-rendering', -7.149966064766881)]

#### Tarefa

Separe o RDD original em dois RDDs: o dos reviews positivos e o dos negativos. Em seguida, use as funções anteriores para construir RDDs que contem os pares (palavra, $\log_{10}\left(c \, / \, T\right)$)

In [12]:
def filtra_categ(rdd, categ):
    operation = rdd.filter(lambda x: x[0] == categ)
    return operation

In [13]:
positive = filtra_categ(rdd, 2)
negative = filtra_categ(rdd, 1)

In [14]:
positive.take(3)

[(2,
  'Stuning even for the non-gamer',
  'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'),
 (2,
  'The best soundtrack ever to anything.',
  "I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."),
 (2,
  'Amazing!',
  'This soundt

In [15]:
negative.take(3)

[(1,
  'Buyer beware',
  'This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon\'s family and friends--or perhaps, by herself! I can\'t imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a "worst book" contest. I can\'t believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on "To Kill a Mockingbird"--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!'),
 (1,
  'The Worst!',
  "A complete waste of time. Typographical errors, poor grammar, and a totally pathetic plot add up to absolutely nothing. I'm embarrassed for this author and very disappointed I actually paid for this book."),
 (1,
  'Oh 

Agora queremos que cada par positivo/negativo, tenha também o seu log(c/T)

In [16]:
positive_ = palavra_contagem(positive)
negative_ = palavra_contagem(negative)

In [17]:
total_positive = positive_.reduce(lambda x, y: ("tudo", x[1] + y[1]))
total_negative = negative_.reduce(lambda x, y: ("tudo", x[1] + y[1]))

In [18]:
positive_log = logaritmo(positive_, total_positive)
negative_log = logaritmo(negative_, total_negative)

In [19]:
positive_log.take(3)

[('out', -2.6616432990632193),
 ('like', -2.493354335745979),
 ('heart-rendering', -6.877611149594514)]

In [20]:
negative_log.take(3)

[('family', -3.9203467150039017),
 ('book', -2.3376666325195163),
 ('purposes.', -5.277100209404035)]

### Tarefa

Use o `.fullOuterJoin()` dos RDDs para construir um RDD unificado, no qual cada item é da forma (palavra, log_prob_positivo, log_prob_negativo). "Baixe" esse resultado final usando `.collect()`.

In [21]:
palavra_concat = positive_log.fullOuterJoin(negative_log).collect()

In [22]:
palavra_concat[0]

('out', (-2.6616432990632193, -2.596022446951644))

Vamos precisar converter para um dicionario. Isso porque é muito mais fácil encontrar a palavra caso ela seja uma chave do dict, se for uma lista, é preciso percorrer toda a lista.

In [23]:
def convert_tupla_dict(tupla):
    dictionary = {}
    for word, values in tupla:
        dictionary[word] = values
    
    return dictionary

In [24]:
palavra_concat_dict = convert_tupla_dict(palavra_concat)

In [25]:
palavra_concat_dict

{'out': (-2.6616432990632193, -2.596022446951644),
 'heart-rendering': (-6.877611149594514, -7.865371916246365),
 'angery.': (-7.831853659033838, -7.689280657190683),
 'age-old': (-6.018940302390982, -6.30307905178989),
 'family': (-3.4798931704033724, -3.9203467150039017),
 'start': (-3.5690467360332745, -3.6622437996551502),
 'willing': (-4.465804449233603, -4.361649309759885),
 'choices.': (-5.239676901637972, -5.309069415479077),
 'Electronics': (-5.845081924767593, -5.551504695877211),
 'gift': (-3.614264444068743, -3.915737992447102),
 'southside': (-6.7904609738756125, -7.166401911910346),
 'guest': (-4.751046854699476, -4.981426721212085),
 'nails,': (-5.738431973871603, -5.746446163420588),
 'cute': (-4.055043551962825, -4.201388461638098),
 'smoother-than-silk': (-8.13288365469782, None),
 'JMM': (-7.6557623999781566, None),
 '24': (-4.609137187886255, -4.498109201462125),
 'Later,': (-5.69831475066362, -5.7667281904293075),
 'fuel': (-5.083665632027637, -5.050458734971291),


Abaixo podemos verificar um comparativo do impacto da mudança:

In [26]:
"family" in palavra_concat

False

In [27]:
"family" in palavra_concat_dict

True

#### Tarefa

Para uma dada string, determine se ela é um review positivo ou negativo usando os RDDs acima. Lembre-se de como funciona o classificador Naive-Bayes: http://stanford.edu/~jurafsky/slp3/slides/7_NB.pdf, consulte tambem suas notas de aula de Ciência dos Dados!

In [28]:
#vamos pegar um teste qualquer presente no dataset
phrase = "A complete waste of time. Typographical errors, poor grammar, and a totally pathetic plot add up to absolutely nothing. I'm embarrassed for this author and very disappointed I actually paid for this book."

In [29]:
all_terms = phrase.split()
all_terms

['A',
 'complete',
 'waste',
 'of',
 'time.',
 'Typographical',
 'errors,',
 'poor',
 'grammar,',
 'and',
 'a',
 'totally',
 'pathetic',
 'plot',
 'add',
 'up',
 'to',
 'absolutely',
 'nothing.',
 "I'm",
 'embarrassed',
 'for',
 'this',
 'author',
 'and',
 'very',
 'disappointed',
 'I',
 'actually',
 'paid',
 'for',
 'this',
 'book.']

In [30]:
def naive_bayes(words, reference):
    #precisamos divir em palavras positivas e palavras negativas e somar a probabilidade total
    positive = 0
    negative = 0
    
    for word in words:
        if word in reference:
            if reference[word][0] != None:
                positive += reference[word][0]
            if reference[word][1] != None:
                negative += reference[word][1]
    
    print(positive)
    print(negative)
    if positive > negative:
        return "positive"

    return "negative"
    

In [31]:
result = naive_bayes(all_terms, palavra_concat_dict)
result

-112.3695899560691
-104.0231864327492


'negative'

### Fase 2

Agora que temos um classificador Naive-Bayes, vamos explorá-lo um pouco:

### Tarefa

Quais são as 100 palavras que mais indicam negatividade, ou seja, onde a diferença entre a probabilidade da palavra no conjunto dos comentários negativos e positivos é máxima? E quais as 100 palavras de maior positividade? Mostre os resultados na forma de *word clouds*.

In [32]:
palavra_concat = positive_log.fullOuterJoin(negative_log)

In [33]:
palavra_rdd = palavra_concat

In [34]:
most_negative = palavra_rdd.filter(lambda x: x[1][0] != None).filter(lambda x: x[1][1] != None).map(lambda x: (x[0], x[1][0], x[1][1], x[1][1] - x[1][0])).takeOrdered(100, lambda x: -x[3])

In [35]:
most_negative

[('Worthless', -7.4339136503618, -4.730080211770613, 2.7038334385911877),
 ('Awful!', -7.831853659033838, -5.194662321022568, 2.63719133801127),
 ('Terrible!', -7.6557623999781566, -5.138644707219792, 2.5171176927583643),
 ('Uninspired', -8.13288365469782, -5.627325813117569, 2.5055578415802504),
 ('Avoid!', -8.13288365469782, -5.6387720110390065, 2.4941116436588127),
 ('Useless.', -8.13288365469782, -5.6413571048735, 2.491526549824319),
 ('Disappointed...', -8.13288365469782, -5.6413571048735, 2.491526549824319),
 ('JUNK!!!', -8.13288365469782, -5.677851195409901, 2.455032459287918),
 ('Junk!', -7.6557623999781566, -5.213125575243041, 2.442636824735115),
 ('worthless!', -8.13288365469782, -5.702508922924438, 2.430374731773381),
 ('Worthless.', -8.13288365469782, -5.72235711599227, 2.4105265387055494),
 ('unfunny,', -8.13288365469782, -5.728651349089958, 2.404232305607861),
 ('Dull,', -7.831853659033838, -5.441307390828877, 2.390546268204961),
 ('Unreliable', -7.530823663369857, -5.180

In [36]:
most_positive = palavra_rdd.filter(lambda x: x[1][0] != None).filter(lambda x: x[1][1] != None).map(lambda x: (x[0], x[1][0], x[1][1], x[1][0] - x[1][1])).takeOrdered(100, lambda x: -x[3])

In [37]:
most_positive

[('Excellent!!!', -5.743717570333287, -8.166401911910345, 2.4226843415770585),
 ('must-read!', -5.923368640155188, -8.166401911910345, 2.2430332717551567),
 ('Inspiring!', -5.939759056343357, -8.166401911910345, 2.226642855566988),
 ('Outstanding!', -5.344008538922402, -7.564341920582383, 2.220333381659981),
 ('Excellent!!', -5.682634546378458, -7.865371916246365, 2.1827373698679065),
 ('Adorable!', -5.993004568296582, -8.166401911910345, 2.173397343613763),
 ('Underrated', -5.415213151695557, -7.564341920582383, 2.1491287688868264),
 ('Bookwatch', -6.025673685049951, -8.166401911910345, 2.1407282268603938),
 ('Fantastic!!', -6.029079933741862, -8.166401911910345, 2.137321978168483),
 ('Excellent!', -4.714914012483082, -6.804674075892753, 2.0897600634096705),
 ('Gem!', -6.079805211214399, -8.166401911910345, 2.086596700695946),
 ('must-have!', -5.7845787916496585, -7.865371916246365, 2.080793124596706),
 ('Insightful!', -6.091490969539594, -8.166401911910345, 2.0749109423707512),
 ('Aw

### Tarefa desafio!

Qual o desempenho do classificador (acurácia)? Para medir sua acurácia:

- Separe os reviews em dois conjuntos: treinamente e teste
- Repita o "treinamento" do classificador com o conjunto de treinamento
- Para cada review do conjunto de teste, determine se é positiva ou negativa de acordo com o classificador
- Determine a acurácia

Esta não é uma tarefa trivial. Não basta fazer um `for` para determinar a classe de cada review de teste: isso demoraria uma eternidade. Você tem que usar variáveis "broadcast" do Spark para enviar uma cópia da tabela de frequencias para cada *core* do executor.

### Tarefa desafio!

Implemente Laplace smoothing

In [39]:
#valores retirados do projeto de cdados (https://github.com/wilgnerl/Projeto1-Cdados)
alfa = 0.01
V = 10e4
def naive_bayes_with_laplace(rdd, alfa, V, T):
    return rdd.map(lambda x: (x[0], np.log( (x[1] + alfa)/(T[1] + V*alfa) ) ) )

In [40]:
#Agora que temos o laplace smoothing, basta reaplicar nos testes que fizemos para verificar a diferença
positive_smoothing = naive_bayes_with_laplace(positive_, alfa, V, total_positive)
negative_smoothing = naive_bayes_with_laplace(negative, alfa, V, total_negative)

In [45]:
positive_smoothing.take(4)

[('out', -6.128667513519069),
 ('like', -5.7411678661236305),
 ('heart-rendering', -15.835736871190655),
 ('book', -5.233246177825735)]

In [46]:
palavra_concat_smooth = positive_smoothing.fullOuterJoin(negative_smoothing)

In [47]:
palavra_concat_smooth.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 48 in stage 36.0 failed 1 times, most recent failure: Lost task 48.0 in stage 36.0 (TID 875) (4ec2a45a0c45 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_11344/4237827204.py", line 5, in <lambda>
TypeError: can only concatenate str (not "float") to str

	at o

In [ ]:
word_dict_smooth = convert_tupla_dict(palavra_concat_smooth)

In [ ]:
#vamos pegar um teste!


## Rubrica de avaliação

- I: groselha, falha crítica, ou não entregou nada
- D: Fez uma tentativa honesta de fazer todos os itens da fase 1, mas tem erros
- C: Fase 1 completa
- B: Fase 2, faltando apenas um desafio
- A: Fase 2 completa